# Hopf Bifurcation Study

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext

import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import dolfin as dfn

from femvf import load, statefile as sf, meshutils
from femvf.models.transient import solid as smd, fluid as fmd
from femvf.signals.solid import make_sig_glottal_width_sharp
from blockarray import h5utils as bh5utils, blockvec as bvec
from vfsig import modal
from vfsig.misc import resample_over_uniform_period

import libhopf
import main_hopf
import libsignal
from lib_main_transient import case_config
import h5utils
from postprocutils import postprocess_case_to_signal

In [ ]:
## Global vars
ZETA = 1e-4
R_SEP = 1.0
Y_GAP = 1e-2
# PSUBS = np.concatenate([np.arange(200, 300, 10), np.arange(300, 1000, 100)])*10
PSUBS = np.arange(550, 650, 10) * 10 
ECOV = 5e3 * 10
# EBODY = 15e3 * 10
EBODY = 5e3 * 10

# Load the transient model
INIT_STATE_TYPE = 'static'

OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_fixed_rsep'
FluidType = fmd.BernoulliMinimumSeparation
# OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_variable_rsep'
# FluidType = fmd.Bernoulli

MESH_NAME = 'BC-dcov5.00e-02-cl1.00'
mesh_path = f'mesh/{MESH_NAME}.xml'
model_tran = load.load_transient_fsi_model(
    mesh_path, None, 
    SolidType=smd.KelvinVoigt, 
    FluidType=FluidType, 
    coupling='explicit'
)

# load the Hopf models
res, dres = main_hopf.setup_models(mesh_path)
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar'])
_props = main_hopf.set_props(res.props.copy(), _region_to_dofs, res)
res.set_props(_props)
dres.set_props(_props)

res_hopf = libhopf.HopfModel(res, dres)

## Post-process nonlinear oscillations (transient simulations)

In [ ]:
## Initialize functions to extract glottal width from saved simulations

# Number of points per period to use in processing
N_PER_PERIOD = 100
proc_gw_tran = make_sig_glottal_width_sharp(model_tran)
proc_gw_hopf = libsignal.make_glottal_width(res_hopf, num_points=N_PER_PERIOD)
def time(f):
    return np.array(f.get_times())

In [ ]:
## Post processing transient simulations
postprocess_fname = f'{OUT_DIR}/data.h5'
signal_to_proc = {
    'glottal_width': proc_gw_tran,
    'time': time}
files = [f'{OUT_DIR}/{case_config(MESH_NAME, psub, ECOV, EBODY)}.h5' for psub in PSUBS]

# `SIGNALS` is a dictionary mapping each `case` and signal name to a time signal
SIGNALS = postprocess_case_to_signal(postprocess_fname, files, model_tran, signal_to_proc)

# with h5py.File(postprocess_fname, mode='r') as f:
#     SIGNALS = h5utils.h5_to_dict(f, {})

In [ ]:
## Resample the transient simulations so they have a fixed number of points per period
cases = list(set([key.split('/')[0] for key in SIGNALS.keys()]))

# NOFFSET controls the portion of the signal to treat as 'steady state'
# -8124, sets the final 8124 samples to be considered a steady state portion
# This portion of the signal is resampled on a periodic basis
NOFFSET = -2**10
NPERIOD = 5


MEASURES = {}
for case in cases:
    # F0 and period are in units of samples
    fund_freq, _, df = modal.estimate_fundamental_mode(
        SIGNALS[f'{case}/glottal_width'][NOFFSET:]
    )
    MEASURES[f'{case}/fund_freq'] = fund_freq
    MEASURES[f'{case}/dfreq'] = df
    
    MEASURES[f'{case}/period'] = fund_freq**-1

RESIGNALS = {}
for case in cases:
    xp = np.arange(SIGNALS[f'{case}/glottal_width'][NOFFSET:].size)
    fp = SIGNALS[f'{case}/glottal_width'][NOFFSET:]
    
    # `num_period` is the number of periods/cycles in the sequence
    # This should always be an integer due to how FFT computes the
    # signal frequencies
    samples_per_period = MEASURES[f'{case}/period']
    num_period = int(round(fp.size/samples_per_period))
    
    x = np.linspace(0, num_period, int(num_period)*N_PER_PERIOD + 1) * samples_per_period
    
    RESIGNALS[f'{case}/glottal_width'] = np.interp(x, xp, fp)

resignals_update = {
    f'{case}/time': 
    np.arange(RESIGNALS[f'{case}/glottal_width'].size)/MEASURES[f'{case}/period']
    for case in cases}

RESIGNALS.update(resignals_update)

In [ ]:
## Post processing Hopf simulation (load the Hopf bifurcation state)
hopf_sim_fpath = f"out/hopf_state.h5"

with h5py.File(hopf_sim_fpath, mode='r') as f:
    hopf_state = bh5utils.read_block_vector_from_group(f)

# Decompose the Hopf state vector to common components
xfp_hopf = hopf_state[:4]
mode_hopf_real, mode_hopf_imag = hopf_state[4:8], hopf_state[8:12]
# mode_hopf = mode_hopf_real + 1j*mode_hopf_imag
omega_hopf = hopf_state['omega'][0]
psub_hopf = hopf_state['psub'][0]

unit_mode_real, unit_mode_imag = libhopf.normalize_eigenvector_amplitude(
    mode_hopf_real, mode_hopf_imag)

## Plots

### Nonlinear oscillations

In [ ]:
## Plot transient simulations
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for psub in PSUBS:
    case = case_config(MESH_NAME, psub, ECOV, EBODY)
    ret = RESIGNALS[f'{case}/time']
    regw = RESIGNALS[f'{case}/glottal_width']
    
    t = SIGNALS[f'{case}/time']
    dt = t[1] - t[0]
    freq = MEASURES[f'{case}/fund_freq'] / dt
    dfreq = MEASURES[f'{case}/dfreq'] / dt
    
    # t = SIGNALS[f'{case}/time']
    # gw = SIGNALS[f'{case}/glottal_width']
    
    ax.plot(ret, regw, label=f"$p_{{sub}}$: {psub/10:.1f} Pa $F_0$: {freq:.1f} +- {dfreq:.1f} Hz", lw=1.0)
    
# ax.set_xlim(60, 70)
# ax.set_xlim(0, 2)

ax.set_xlabel("Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend(loc='lower left', bbox_to_anchor=(0, 1))

fig.tight_layout()
fig.savefig(f'fig/transient_glottal_width.png')

### Small amplitude oscillations

In [ ]:
## Plot Hopf simulations
fig, ax = plt.subplots(1, 1)

for ampl in np.linspace(0, 20000.0, 5):
    gw = proc_gw_hopf(
        hopf_state.to_mono_ndarray(),
        np.array([ampl, 0.0]))
    ax.plot(gw, label=f"Amplitude {ampl:.2e}")
    
ax.set_xlabel(f"Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend()

fig.tight_layout()
fig.savefig("fig/onset_gw_vs_amplitude.png", dpi=250)

In [ ]:
## Plot the onset mode shape
import dolfin as dfn
XREF = res.solid.XREF.vector()

NPHASE = 5
phases = 2*np.pi*np.linspace(0, 1, NPHASE+1)[:-1]

VERT_TO_VECDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])
CELLS = res.solid.forms['mesh.mesh'].cells()

fig, axs = plt.subplots(1, NPHASE, sharey=True, sharex=True, figsize=(10, 5))
for ax, phase in zip(axs, phases):
    ampl = 5e4*np.exp(1j*phase)
    mode_hopf_dof_order = (
        XREF[:] + hopf_state['u'] 
        + np.real(
            ampl * (
                hopf_state['u_mode_real'] 
                + 1j*hopf_state['u_mode_imag']
            )
        )
    )
    
    mode_hopf_vert_order = mode_hopf_dof_order[VERT_TO_VECDOF].reshape(-1, 2)
    # print(mode_hopf_vert_order.shape)
    ax.triplot(*mode_hopf_vert_order.T, triangles=CELLS)
    ax.set_title(f"${phase/np.pi:.2f}\pi$ rad")
    
for ax in axs:
    ax.set_aspect('equal')
    ax.set_xlabel("x [cm]")
axs[0].set_ylabel("y [cm]")
    
fig.tight_layout()
fig.savefig(f'fig/hopf_mode_shape.png', dpi=200)

In [ ]:
def plot_onset_mode(axs):

### Compare nonlinear and small-amplitude oscillations

In [ ]:
## Plot Transient vs Hopf simulations
SCALE = np.array([1e4, 1])

def sqr_err(sc_ampl_phase, gw_ref):
    """
    Return the squared error between small amplitude and reference glottal widths
    """
    ampl_phase = sc_ampl_phase*SCALE
    gw_hopf = proc_gw_hopf(
        hopf_state.to_mono_ndarray(), 
        ampl_phase)
    
    # Weight the reference glottal width measurements
    # linearly with glottal width; larger glottal widths
    # get higher weights
    weight = gw_ref/jnp.abs(gw_ref.max()) 
    
    err = weight*(gw_hopf-gw_ref)**2
    return jnp.sum(err)

from scipy import optimize

fig, ax = plt.subplots(1, 1)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
psubs = np.array([600, 700, 800, 900])*10
psubs = PSUBS
# psubs = PSUBS[0::2]

cases = [case_config(MESH_NAME, psub, ECOV, EBODY) for psub in psubs]
for ii, (psub, color, case) in enumerate(zip(psubs, colors, cases)):
    # For each case, load the transient simulation glottal width 
    t = RESIGNALS[f'{case}/time'][-N_PER_PERIOD:]
    gw_tran = RESIGNALS[f'{case}/glottal_width'][-N_PER_PERIOD:]
    
    # Calculate an optimal [phase, amplitude] for the onset glottal width
    # to minimize the difference from the onset glottal width
    ampl_phase0 = np.array([5e3, 0.0])
    opt_res = optimize.minimize(sqr_err, ampl_phase0/SCALE, args=(gw_tran,), jac=jax.grad(sqr_err, 0), method='BFGS')
    ampl_phase = opt_res['x'] * SCALE
    # print(opt_res)
    
    # Compute the onset glottal width with the optimal phase and amplitude
    gw_hopf0 = proc_gw_hopf(
        hopf_state.to_mono_ndarray(),
        ampl_phase0)
    
    gw_hopf = proc_gw_hopf(
        hopf_state.to_mono_ndarray(),
        ampl_phase)
    
    ax.plot(t, gw_tran, color=color, ls='-')
    ax.plot(t, gw_hopf, color=color, ls='-.')
    ax.plot(t, gw_hopf0, color=color, ls=':')
    
ax.set_xlabel("t/T")
ax.set_ylabel("Glottal width [cm]")

from matplotlib.lines import Line2D

f0s = []
for case in cases:
    _t = SIGNALS[f'{case}/time']
    dt = _t[1]-_t[0]
    f0s.append((MEASURES[f'{case}/period'] * dt)**-1)

# Make a legend describing the relationship 
# ls -> model type
# psub -> color
_model_type_ls = ['-', '-.']
_model_type_lines = [Line2D([0], [0], color='k', ls=ls) for ls in _model_type_ls] 
_model_type_labels = ["Transient", f"Onset {omega_hopf/2/np.pi:.1f} Hz {psub_hopf/10:.1f} Pa"]

_psub_colors = colors[:len(psubs)]
_psub_lines = [Line2D([0], [0], color=color) for color in _psub_colors]
_psub_labels = [f"{psub/10:.0f} Pa, {f0:.1f} Hz" for psub, f0 in zip(psubs, f0s)]
ax.legend(
    _model_type_lines+_psub_lines,
    _model_type_labels+_psub_labels,
    loc=(0, 1)
)

fig.savefig('fig/transient_vs_onset_gw.png', dpi=200)

### Optimization results

In [ ]:
# Load mesh and DOF map information for plotting data on the mesh
TRI = res.solid.forms['mesh.mesh'].coordinates()
X, Y = TRI[:, 0], TRI[:, 1]
CELLS = res.solid.forms['mesh.mesh'].cells()

# These indices select DOFs from a scalar function in vertex-order
IDX_VERT = dfn.vertex_to_dof_map(res.solid.forms['fspace.scalar'])

In [ ]:
# Load optimization history
opt_fname = 'opt_hist'
# opt_fname = 'opt_hist_BC-dcov5.00e-02-cl1.00_psub5.7000e+03_ecov5.0000e+04_ebody5.0000e+04'
with h5py.File(f'out/{opt_fname}.h5', mode='r') as f:
    objs = f['objective'][:]
    grads = [bh5utils.read_block_vector_from_group(f['grad'], nn) for nn in range(len(objs))]
    params = [bh5utils.read_block_vector_from_group(f['parameters'], nn) for nn in range(len(objs))]
    hopf_states = [bh5utils.read_block_vector_from_group(f['hopf_state'], nn) for nn in range(len(objs))]
    
its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([hopf_state['psub'][0] for hopf_state in hopf_states])
onset_fs = np.array([hopf_state['omega'][0] for hopf_state in hopf_states])

#### Plot a summary of optimization results

In [ ]:
# Indicate a specific iteration to plot parameters for
N = -4
grad = grads[N]
param = params[N]
hopf_state = hopf_states[N]

In [ ]:
print(param.bshape)
print(res.props.bshape)

In [ ]:
## Compute the spectrum for plotting
# Set the dynamical system residual properties + subglottal pressure
res_hopf.set_state(bvec.convert_subtype_to_petsc(hopf_state))
omegas, *_eigvecs = libhopf.solve_linear_stability(res_hopf.res, hopf_state[res_hopf.labels_state])

print(omegas)

In [ ]:
## Plot a summary of the objective function history and parameters at specific iterations
fig = plt.figure(figsize=(6, 6))
gs = gridspec.GridSpec(3, 2, figure=fig)

# Plot the objective function/gradient norm history
ax_obj = fig.add_subplot(gs[0, :])
ax_grad = fig.add_subplot(gs[1, :])
ax_obj.sharex(ax_grad)
ax_obj.xaxis.set_tick_params(labelbottom=False)

ax = ax_obj
ax.plot(its, objs)
ax.axvline([its[N]], color='k', ls='-.', alpha=0.5)

ax.set_xlabel("Iteration")
ax.set_ylabel("Functional")

ax.set_ylim([0.8*objs.min(), 1.2*objs[0]])

ax = ax_grad
ax.plot(its, grads_norm)
ax.axvline([its[N]], color='k', ls='-.', alpha=0.5)
# ax.set_ylim([0.8*grads_norm[0], 1.2*grads_norm[0]])
ax.set_xlim(its[[0, -1]])
ax.set_ylabel("||grad||")
ax.set_yscale('log')

# Plot the current parameter guess
ax_mod = fig.add_subplot(gs[2, 0])
ax = ax_mod

mappable = ax.tripcolor(X, Y, CELLS, param['emod'][IDX_VERT]/10/1e3)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")

cbar = fig.colorbar(mappable, ax=ax)
cbar.ax.set_ylabel("$E$ [kPa]")
ax.set_aspect(1)

# Plot the current gradient
ax_grad = fig.add_subplot(gs[2, 1])
ax = ax_grad

mappable = ax.tripcolor(X, Y, CELLS, grad['emod'][IDX_VERT]/10/1e3)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")

cbar = fig.colorbar(mappable, ax=ax)
cbar.ax.set_ylabel("$dE$ [Pa/kPa]")

ax.set_aspect(1)

fig.tight_layout()
fig.savefig(f'opt_summary_for_{opt_fname}.png', dpi=200)

In [ ]:
## Plot a summary of the onset pressure and frequency for the optimization history
fig = plt.figure(figsize=(6, 4))
gs = gridspec.GridSpec(2, 1, figure=fig)

# Plot the objective function history
ax_onp = fig.add_subplot(gs[0, :])
ax_onf = fig.add_subplot(gs[1, :])
ax_onp.xaxis.set_tick_params(labelbottom=False)

ax_onp.plot(its, np.array(onset_ps)/10)
ax_onf.plot(its, np.array(onset_fs)/2/np.pi)

ax_onp.set_ylabel("Onset pressure [Pa]")
ax_onf.set_ylabel("Onset frequency [Hz]")
ax_onf.set_xlabel("Iteration")

fig.tight_layout()
fig.savefig(f'opt_onset_{opt_fname}.png', dpi=200)

In [ ]:
# Plot the spectrum
fig, ax = plt.subplots(1, 1)
# ax_spec = fig.add_subplot(gs[2, 0])
# ax = ax_spec
ax.scatter(omegas.real, omegas.imag)
ax.axhline(0, color='k')
ax.axvline(0, color='k')

ax.set_xlabel("Real")
ax.set_ylabel("Imag")
ax.set_xlim(-100, 50)

fig.tight_layout()

In [ ]:
## Plot the final mode shape
import dolfin as dfn
XREF = res.solid.XREF.vector()

NPHASE = 5
phases = 2*np.pi*np.linspace(0, 1, NPHASE+1)[:-1]

VERT_TO_VECDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])
CELLS = res.solid.forms['mesh.mesh'].cells()

fig, axs = plt.subplots(1, NPHASE, sharey=True, sharex=True, figsize=(10, 5))
for ax, phase in zip(axs, phases):
    ampl = 5e4*np.exp(1j*phase)
    mode_hopf_dof_order = (
        XREF[:] + hopf_state['u'] 
        + np.real(
            ampl * (
                hopf_state['u_mode_real'] 
                + 1j*hopf_state['u_mode_imag']
            )
        )
    )
    
    mode_hopf_vert_order = mode_hopf_dof_order[VERT_TO_VECDOF].reshape(-1, 2)
    # print(mode_hopf_vert_order.shape)
    ax.triplot(*mode_hopf_vert_order.T, triangles=CELLS)
    ax.set_title(f"${phase/np.pi:.2f}\pi$ rad")
    
for ax in axs:
    ax.set_aspect('equal')
    ax.set_xlabel("x [cm]")
axs[0].set_ylabel("y [cm]")
    
fig.tight_layout()
fig.savefig(f'fig/hopf_mode_shape_optimization.png', dpi=200)

#### Inverse problem

In [ ]:
## Plot a comparison of the estimated glottal width vs measured glottal width
gt_dir = 'out/zeta1.00e-04_rsep1.0_ygap1.00e-02_initstatic_fixed_rsep'
gt_fname = 'BC-dcov5.00e-02-cl1.00_psub5.7000e+03_ecov5.0000e+04_ebody5.0000e+04'
with sf.StateFile(model_tran, f'{gt_dir}/{gt_fname}.h5', mode='r') as f:
    t_gt = f.get_times()
    gw_gt = proc_gw_tran(f)

In [ ]:
std_gw = (0.1/5) / (np.maximum(gw_gt, 0.0) / gw_gt.max())

In [ ]:
gw_hopf = proc_gw_hopf(
    hopf_state.to_mono_ndarray(),
    param[-2:].to_mono_ndarray()
)
f_hopf = onset_fs[-1]/2/np.pi

t_hopf = 1/np.abs(f_hopf)*np.linspace(-1.0, 0.0, gw_hopf.size+1)[:-1] + t_gt[-1]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3))

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

gt_color = colors[0]
ax.plot(t_gt, gw_gt, label="Ground truth")
ax.plot(t_gt, gw_gt + std_gw, color=gt_color, ls=':', label="+-stdev")
ax.plot(t_gt, gw_gt - std_gw, color=gt_color, ls=':')
ax.plot(t_hopf, gw_hopf, color=colors[1], label="Estimated")

ax.set_ylabel("Glottal width [cm]")
ax.set_xlabel("Time [s]")
ax.set_xlim(t_hopf[[0, -1]])
ax.set_ylim([
    1.5*min(gw_hopf.min(), gw_gt.min()), 
    1.5*max(gw_hopf.max(), gw_gt.max())
])

ax.legend()

fig.savefig('fig/estimated_vs_gt_gw.png', dpi=200)

## Debugging

For the case where we minimize onset pressure while maintaining a constant onset frequency, the optimization encounters a problem. This is illustrated below for the history file 'OPT_DEBUG.h5' which started from a uniform initial guess of 5 kPa.

At some of the last iterations, optimization progress stalls and the optimization routine fails with a zero size step on a line search. A likely reason why this occurs can be seen by plotting the maximum real eigenvalue as a function of subglottal pressure. The minimal subglottal pressure of 0.38 kPa occurs due to a very sharp peak in the profile. It is likely that as parameters change in the line search, this small peak disappears so that onset jumps to the next onset pressure of around 0.5 kPa.

In [ ]:
# Load optimization history
opt_fname = 'OPT_DEBUG_1'
with h5py.File(f'out/{opt_fname}.h5', mode='r') as f:
    objs = f['objective'][:]
    grads = [bh5utils.read_block_vector_from_group(f['grad'], nn) for nn in range(len(objs))]
    params = [bh5utils.read_block_vector_from_group(f['parameters'], nn) for nn in range(len(objs))]
    hopf_states = [bh5utils.read_block_vector_from_group(f['hopf_state'], nn) for nn in range(len(objs))]
    
its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([hopf_state['psub'][0] for hopf_state in hopf_states])
onset_fs = np.array([hopf_state['omega'][0] for hopf_state in hopf_states])

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True)

axs[0].plot(its, objs)
axs[0].set_ylabel("Objective")

axs[1].plot(its, onset_ps)
axs[1].set_ylabel("Onset pressure")

axs[2].plot(its, np.abs(onset_fs))
axs[2].set_ylabel("Onset freq.")

axs[2].set_xlabel("")

### Compute a line search along a given direction

In [ ]:
# compute functionals along the line from N -> N+1
N = 25
res_hopf.set_state(bvec.convert_subtype_to_petsc(hopf_states[N]))
res_hopf.set_props(bvec.convert_subtype_to_petsc(params[N]))

xhopf_n, info = libhopf.solve_hopf_newton(res_hopf, res_hopf.state)
print(info)
print(xhopf_n['psub'][0])

### Compute the spectrum at a specific iteration

In [ ]:
# Specify the parameter set from the optimization to use
N = 26
res.set_props(bvec.convert_subtype_to_petsc(params[N][:-2]))

In [ ]:
## Plot the spectrum at the onset pressure

def plot_spectrum(fig, ax, N):
    res.set_props(bvec.convert_subtype_to_petsc(params[N][:-2]))
    res.control['psub'][0] = onset_ps[N]
    res.set_control(res.control)

    xfp_n = hopf_states[N][res_hopf.labels_state]

    omegas, eigvecs_real, eigvecs_imag = libhopf.solve_linear_stability(res, xfp_n)
    
    ax.scatter(omegas.real, omegas.imag)
    ax.axhline(0, color='k')
    ax.axvline(0, color='k')

    ax.set_xlabel("$\Re(\omega)$")
    ax.set_ylabel("$\Im(\omega)$")
    ax.set_title(f"$P_\mathrm{{onset}}$={onset_ps[N]/10:.1f} Pa")
    ax.set_xlim(-50, 10)
    ax.set_ylim(-1200, 1200)

    fig.tight_layout()
    return fig, ax

# for n in range(0, 27):
#     fig, ax = plt.subplots(1, 1)
#     fig, ax = plot_spectrum(fig, ax, n)
#     fig.savefig(f"fig/Spectrum{n}.png")
#     plt.close(fig)
    
fig, ax = plt.subplots(1, 1)
fig, ax = plot_spectrum(fig, ax, N)

In [ ]:
## Compute the most unstable eigenvalue over a range of subglottal pressures
# This gives a rough idea at which subglottal pressure(s) (if any) onset will occur

psubs = np.arange(300, 600, 2.5)*10
# psubs = np.arange(0, 1500, 100)*10
omegas_max = np.zeros(psubs.shape, dtype=complex)

xfp_0 = res.state.copy()
xfp_0.set(0.0)
for ii, psub in enumerate(psubs):
    res.control['psub'][0] = psub
    res.set_control(res.control)

    xfp_n, info = libhopf.solve_fixed_point(res, xfp_0, newton_params={'max_iter': 20})
    # print(f"Solving for fixed point took {info['num_iter']} iterations. Abs err {info['abs_errs'][-1]}")
    omegas, eigvecs_real, eigvecs_imag = libhopf.solve_linear_stability(res, xfp_n)
    
    idx_max = np.argmax(omegas.real)
    omegas_max[ii] = omegas[idx_max]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 4), sharex=True)

# idx = np.logical_and(psubs >= 2000, psubs <= 6000)
idx = np.ones(psubs.shape, dtype=bool)

axs[0].plot(psubs[idx], omegas_max[idx].real)
axs[0].axhline(0, color='k')
axs[0].set_ylabel("$real \; \omega$ $[rad/s]$")


axs[1].plot(psubs[idx], omegas_max[idx].imag)
axs[1].axhline(0, color='k')
axs[1].set_ylabel("$imag \; \omega$ $[rad/s]$")

# axs[1].set_xlim(3500, 4000)
# axs[0].set_ylim()


axs[1].set_xlabel("$P_{sub}$ $[Pa]$")